<a href="https://colab.research.google.com/github/adarshgovi/adarshgovi/blob/main/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import cv2

import matplotlib.pyplot as plt
import imutils
import numpy

In [143]:
def findLine(vidFrame):
  croppedFrame=vidFrame[0:240][200:320]
  mask=cv2.cvtColor(croppedFrame,cv2.COLOR_BGR2GRAY)
  blurred=cv2.GaussianBlur(mask,(5,5),0)
  thresh=cv2.threshold(blurred,60,255,cv2.THRESH_BINARY)[1]
  invertedMask=cv2.bitwise_not(thresh)
  contours=cv2.findContours(invertedMask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  contours=imutils.grab_contours(contours)

  if len(contours)>1:
    for c in contours:
      M=cv2.moments(c)
      if (M["m00"]>0):
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # draw the contour and center of the shape on the image
        cv2.drawContours(vidFrame, [c], -1, (0, 255, 0), 2)
        cv2.circle(vidFrame, (cX, cY), 7, (255, 255, 255), -1)
        cv2.putText(vidFrame, "center", (cX - 20, cY +180), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
      else:
        cv2.putText(vidFrame,"div 0 error from moments", (160,120),cv2.FONT_HERSHEY_SIMPLEX,10,(255,255,255),2)

      plt.imshow(vidFrame)

  elif len(contours)>0:
    biggestContour=max(contours,key=cv2.contourArea)
    moments=cv2.moments(biggestContour)
    centerX=int(moments["m10"]/moments["m00"])
    cv2.drawContours(vidFrame,[biggestContour],-1,(0,255,0),2)
    cv2.circle(vidFrame,(centerX,210),20,(255,0,0),-1)
 
  else:
    cv2.putText(vidFrame, "center", (160,120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

  return vidFrame




In [160]:
# Open a video file to read from it:
video_name = "/content/drive/MyDrive/Lab02/raw_video_feed.mp4"
video_reader = cv2.VideoCapture(video_name)
ret, frame = video_reader.read()

# Create a video file to write to:
shape = frame.shape
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
video_writer = cv2.VideoWriter('output_4.mp4', fourcc, 20, (shape[1], shape[0]))
loopCount=0
broken=True

while ret and broken:
  vidFrame=frame
  croppedFrame=frame[0:240][200:320] #only look at the lower portion of the image since we want to follow the line there
  mask=cv2.cvtColor(croppedFrame,cv2.COLOR_BGR2GRAY) #mask with gray to simplify thresholding
  blurred=cv2.GaussianBlur(mask,(5,5),0) #apply a blur to reduce the possibility of noise on edges
  thresh=cv2.threshold(blurred,60,255,cv2.THRESH_BINARY)[1] #take only the most prominent shapes(i.e. darkest)
  invertedMask=cv2.bitwise_not(thresh) #invert the mask because of how find contours works
  contours=cv2.findContours(invertedMask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) #locate the contours in the image
  contours=imutils.grab_contours(contours)

  # if len(contours)>1:
  #   for c in contours:
  #     M=cv2.moments(c)
  #     if (M["m00"]>0):
  #       cX = int(M["m10"] / M["m00"])
  #       cY = int(M["m01"] / M["m00"])
  #       # draw the contour and center of the shape on the image
  #       cv2.drawContours(vidFrame, [c], -1, (0, 255, 0), 2)
  #       cv2.circle(vidFrame, (cX, cY), 7, (255, 255, 255), -1)
  #       cv2.putText(vidFrame, "center", (cX - 20, cY +180), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
  #     else:
  #       cv2.putText(vidFrame,"div 0 error from moments", (160,120),cv2.FONT_HERSHEY_SIMPLEX,10,(255,255,255),2)

  #     plt.imshow(vidFrame)
  #     broken=False

  if len(contours)>0:
    biggestContour=max(contours,key=cv2.contourArea) #find the biggest contour 
    moments=cv2.moments(biggestContour) #calculate the moment of inertia from the biggest contour
    centerX=int(moments["m10"]/moments["m00"]) #locate the x coordinate of the center of mass
    #cv2.drawContours(vidFrame,[biggestContour],-1,(0,255,0),2) # previously outlined the contour, but only needed for testing
    prevCenter=centerX #store the most recent center value incase it is needed if the line is lost later
    cv2.circle(vidFrame,(centerX,210),20,(255,0,0),-1) #insert the circle at the location that was determined as the center
 
  else:
    cv2.circle(vidFrame,(prevCenter,210),20,(255,0,0),-1) #if we don't find a contour, look at history, ideally should use some PID algorithm.
  
  video_writer.write(vidFrame)
  ret, frame=video_reader.read()

video_reader.release()
video_writer.release()
loopCount

0